In [ ]:
import pandas as pd
import numpy as np
import hddm
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from kabuki.analyze import gelman_rubin

# Generate and estimate

In [ ]:
#set number of synthetic subjects and amount of trial to generate per subject
subjects = 4
trials = 60

In [ ]:
#create array with all combinations of three values for a, t, scaler and alpha
x = np.array([(a, t, scaler, alpha)
              for a in np.linspace(1.5,2.5,num=3) 
              for t in np.linspace(0.3,0.5,num=3) 
              for scaler in np.linspace(1.5,3,num=3) 
              for alpha in np.linspace(0.15,0.45,num=3)
              ])
f = pd.DataFrame(data=x,columns=['a','t', 'scaler', 'alpha'])

In [ ]:
#generate data, estimate model and save traces and convergence info for all combinations of models
#this will take a while, so sending these as jobs on a cluster is recommended
for i in range(f.shape[0]):
    print(i)
    data = hddm.generate.gen_rand_rlddm_data(a=f.a[i],alpha=f.alpha[i],scaler=f.scaler[i],t=f.t[i],size=trials,subjs=subjects,p_upper=0.75,p_lower=0.25)
    data['q_init'] = 0.5
    models = []
    for a in range(3):  
        m = hddm.HDDMrl(data=data)
        m.sample(30,burn=15,dbname='traces.db',db='pickle')
        models.append(m)
    gelman = gelman_rubin(models)
    gelman = pd.DataFrame.from_dict(gelman,orient='index')
    convergencename = 'convergence_model%s.csv'%(i)
    gelman.to_csv(convergencename)
    traces = m.get_traces()
    filename = 'traces_model%s.csv'%(i)
    traces.to_csv(filename)

# Check convergence

In [ ]:
gelmans = hddm.load_csv('convergence_model0.csv').T
gelmans.columns = gelmans.iloc[0]
gelmans = gelmans.drop('Unnamed: 0',axis=0)
conv = gelmans

for i in range(1,f.shape[0]):
    filename = 'convergence_model%i.csv' %(i)
    gelmans = hddm.load_csv(filename).T
    gelmans.columns = gelmans.iloc[0]
    gelmans = gelmans.drop('Unnamed: 0',axis=0)
    conv = conv.append(gelmans)            
conv = conv.apply(pd.to_numeric)

In [ ]:
#proportion parameters that did not converge when not using find_starting and sample at 3000 and burnin 1500
conv = pd.melt(conv, var_name="Parameter", value_name="Value")
np.mean(conv['Value']>1.1)

In [ ]:
conv.describe()

# Plot parameter recovery

In [ ]:
traces = hddm.load_csv('traces_model0.csv')
tg = traces[traces.columns.drop(list(traces.filter(regex='subj')))]
means = pd.DataFrame(tg.mean(axis=0)).T
means.append(means)

In [ ]:
# Creating an empty Dataframe with column names only
means = pd.DataFrame(columns=['Unnamed: 0','a','a_std','alpha','alpha_std','t','t_std','v','v_std','cor_at'])

for i in range(1,f.shape[0]):
    filename = 'traces_model%i.csv' %(i)
    traces = hddm.load_csv(filename)
    tg = traces[traces.columns.drop(list(traces.filter(regex='subj')))]
    summ = pd.DataFrame(tg.mean(axis=0)).T
    summ['cor_at'] = np.corrcoef(tg['a'],tg['t'])[1,0]
    means = means.append(summ)
means.columns = ['trace','e_a','e_a_std','e_alpha','e_alpha_std','e_t','e_t_std','e_v','e_v_std','cor_at']
means['e_alphaT'] = np.exp(means['e_alpha'])/(1+np.exp(means['e_alpha']))

In [ ]:
means.describe()

In [ ]:
#make in long format to plot all in one figure
f.reset_index(drop=True, inplace=True)
means.reset_index(drop=True, inplace=True)
a=pd.DataFrame({'sim':f['a'],'parameter':'a','est':means['e_a']})
t=pd.DataFrame({'sim':f['t'],'parameter':'t','est':means['e_t']})
alpha=pd.DataFrame({'sim':f['alpha'],'parameter':'alpha','est':means['e_alphaT']})
scaler=pd.DataFrame({'sim':f['scaler'],'parameter':'scaler','est':means['e_v']})
long = a
long = long.append(t)
long = long.append(scaler)
long = long.append(alpha)
long['error'] = np.absolute(long['est']-long['sim'])
long.head()

In [ ]:
#plot absolute error
g = sns.catplot(x='parameter',y='error',sharey=False,data=long)
g.savefig('absolute_error.pdf')

In [ ]:
def plot_hline(y,**kwargs):
    data = kwargs.pop("data") #get the data frame from the kwargs
    plt.axhline(y=y, c='black',linestyle='-',zorder=-1) #zorder places the line underneath the other points

In [ ]:
sns.set(font_scale=1.5)
sns.set_style("white")

In [ ]:
#plot NDT
g = sns.catplot(x='sim',y='est',data=long[long.parameter=='t'])
g.map_dataframe(plot_hline,y=0.3)
g.map_dataframe(plot_hline,y=0.4)
g.map_dataframe(plot_hline,y=0.5)
plt.ylim(0.25,0.55)
g.fig.suptitle("Non-decision time")
g.set_axis_labels("Simulated NDT", "Estimated NDT")

In [ ]:
#plot Decision threshold
g = sns.catplot(x='sim',y='est',data=long[long.parameter=='a'])
g.map_dataframe(plot_hline,y=1.5)
g.map_dataframe(plot_hline,y=2)
g.map_dataframe(plot_hline,y=2.5)
g.fig.suptitle("Decision threshold")
g.set_axis_labels("Simulated threshold", "Estimated threshold")

In [ ]:
#plot alpha
g = sns.catplot(x='sim',y='est',data=long[long.parameter=='alpha'])
g.map_dataframe(plot_hline,y=0.15)
g.map_dataframe(plot_hline,y=0.3)
g.map_dataframe(plot_hline,y=0.45)
g.fig.suptitle("Learning rate")
g.set_axis_labels("Simulated alpha", "Estimated alpha")

In [ ]:
#plot scaling
g = sns.catplot(x='sim',y='est',data=long[long.parameter=='scaler'])
g.map_dataframe(plot_hline,y=1.5)
g.map_dataframe(plot_hline,y=2.25)
g.map_dataframe(plot_hline,y=3)
g.fig.suptitle("Scaling drift rate")
g.set_axis_labels("Simulated scaling", "Estimated scaling")